In [ ]:
import os
import re
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import time
from collections import Counter # Counter tidak digunakan secara aktif, tapi bisa berguna nanti

# --- Konfigurasi ---
NAMA_FOLDER_INPUT = 'input'
# Folder untuk hasil akhir yang berisi perbandingan
NAMA_FOLDER_OUTPUT_COMPARISON_RELATIF = 'output/sastrawi_comparison'
# Folder untuk hasil cleaning saja (opsional, bisa dihapus jika tidak perlu)
NAMA_FOLDER_OUTPUT_CLEANED_RELATIF = 'output/cleaned'
# --- Akhir Konfigurasi ---

base_dir = os.getcwd()
print(f"Menggunakan direktori kerja saat ini sebagai basis: {base_dir}")
input_folder_path = os.path.join(base_dir, NAMA_FOLDER_INPUT)
output_folder_comparison_path = os.path.join(base_dir, NAMA_FOLDER_OUTPUT_COMPARISON_RELATIF)
output_folder_cleaned_path = os.path.join(base_dir, NAMA_FOLDER_OUTPUT_CLEANED_RELATIF)

# --- Fungsi untuk Membersihkan Teks (Mempertahankan Newline) ---
def bersihkan_teks_preserve_lines(teks):
    lines = teks.splitlines()
    cleaned_lines = []
    for line in lines:
        line = line.lower()
        line = re.sub(r"\d+", "", line)
        tanda_baca_escaped = re.escape(string.punctuation)
        line = re.sub(r'[' + tanda_baca_escaped + ']', '', line)
        line = re.sub(r'[ \t]+', ' ', line)
        line = line.strip()
        # Menjaga struktur paragraf, bahkan baris kosong jika hanya berisi whitespace setelah clean
        # Jika ingin menghapus baris yang jadi kosong total: if line: cleaned_lines.append(line)
        cleaned_lines.append(line) # Mempertahankan baris (meskipun kosong setelah clean)
    # Gabung kembali, pastikan ada newline di akhir jika teks asli punya
    hasil = '\n'.join(cleaned_lines)
    # Jika teks asli diakhiri newline, tambahkan kembali
    if teks.endswith('\n') and not hasil.endswith('\n'):
         hasil += '\n'
    return hasil
# --- Akhir Fungsi Cleaning ---

# 1. Inisialisasi Stemmer Sastrawi
print("Menginisialisasi stemmer Sastrawi...")
factory = StemmerFactory()
stemmer = factory.create_stemmer()
print("Stemmer siap.")

# 2. Buat folder output
# Folder untuk perbandingan WAJIB dibuat
if not os.path.exists(output_folder_comparison_path):
    os.makedirs(output_folder_comparison_path)
    print(f"Folder output perbandingan dibuat: {output_folder_comparison_path}")
else:
    print(f"Folder output perbandingan sudah ada: {output_folder_comparison_path}")

# Folder untuk cleaned saja (OPSIONAL) - Jika tidak mau, komentari bagian ini
if not os.path.exists(output_folder_cleaned_path):
    os.makedirs(output_folder_cleaned_path)
    print(f"Folder output cleaned-only dibuat: {output_folder_cleaned_path}")
else:
    print(f"Folder output cleaned-only sudah ada: {output_folder_cleaned_path}")


# Variabel Evaluasi TOTAL (untuk console)
unique_words_cleaned_total = set()
unique_words_stemmed_total = set()
total_words_cleaned_all_files = 0
total_words_stemmed_all_files = 0

# 3. Proses setiap file
print(f"\nMemulai proses cleaning dan stemming dari folder: {input_folder_path}")
start_time = time.time()

try:
    if not os.path.isdir(input_folder_path):
        raise FileNotFoundError(f"Folder input '{input_folder_path}' tidak ditemukan.")

    list_file = os.listdir(input_folder_path)
    processed_files = 0
    skipped_files = 0

    if not list_file:
        print("Folder input kosong.")

    for filename in list_file:
        input_file_path = os.path.join(input_folder_path, filename)

        if os.path.isfile(input_file_path) and filename.lower().endswith('.txt'):
            # Path untuk file output yang berisi perbandingan
            output_file_comparison_path = os.path.join(output_folder_comparison_path, filename)
            # Path untuk file output cleaned-only (opsional)
            output_file_cleaned_path = os.path.join(output_folder_cleaned_path, filename)

            print(f"  -> Memproses file: {filename}...")

            try:
                with open(input_file_path, 'r', encoding='utf-8', errors='ignore') as f_in:
                    original_text = f_in.read()

                # Lakukan Cleaning Teks
                cleaned_text = bersihkan_teks_preserve_lines(original_text)

                # --- Simpan Teks yang Hanya Dibersihkan (OPSIONAL) ---
                # Jika tidak butuh file cleaned terpisah, komentari/hapus blok ini
                with open(output_file_cleaned_path, 'w', encoding='utf-8') as f_clean:
                    f_clean.write(cleaned_text)
                # --- Akhir blok opsional ---

                # Lakukan stemming pada teks yang sudah dibersihkan
                stemmed_text = stemmer.stem(cleaned_text)

                # --- Hitung Statistik UNTUK FILE INI ---
                cleaned_words_list = cleaned_text.split()
                stemmed_words_list = stemmed_text.split()

                count_cleaned_words = len(cleaned_words_list)
                count_unique_cleaned = len(set(cleaned_words_list))
                count_stemmed_words = len(stemmed_words_list)
                count_unique_stemmed = len(set(stemmed_words_list))
                # --- Akhir Hitung Statistik File Ini ---


                # --- Buat Konten untuk File Output Perbandingan ---
                file_header = f"""=============================================
FILE: {filename} - STATISTIK
=============================================
Jumlah Kata (Setelah Cleaning): {count_cleaned_words}
Jumlah Kata Unik (Setelah Cleaning): {count_unique_cleaned}
Jumlah Kata (Setelah Stemming): {count_stemmed_words}
Jumlah Kata Unik (Setelah Stemming): {count_unique_stemmed}
=============================================

"""
                separator_cleaned = "\n--- TEKS SETELAH CLEANING (SEBELUM STEMMING) ---\n"
                separator_stemmed = "\n\n--- TEKS SETELAH STEMMING ---\n" # Tambah newline ekstra

                final_comparison_content = (
                    file_header +
                    separator_cleaned +
                    cleaned_text +
                    separator_stemmed +
                    stemmed_text
                )
                # --- Akhir Pembuatan Konten ---


                # Tulis hasil perbandingan ke file output
                with open(output_file_comparison_path, 'w', encoding='utf-8') as f_comparison:
                    f_comparison.write(final_comparison_content)


                # Update Statistik TOTAL (untuk console)
                unique_words_cleaned_total.update(set(cleaned_words_list))
                unique_words_stemmed_total.update(set(stemmed_words_list))
                total_words_cleaned_all_files += count_cleaned_words
                total_words_stemmed_all_files += count_stemmed_words

                processed_files += 1

            except Exception as e:
                print(f"     ERROR saat memproses file {filename}: {e}")
        else:
            if os.path.isfile(input_file_path):
                 print(f"  -> Melewati file non-txt: {filename}")
            else:
                 print(f"  -> Melewati item yang bukan file: {filename}")
            skipped_files += 1

    end_time = time.time()
    total_time = end_time - start_time

    print(f"\nProses cleaning dan stemming selesai.")
    print(f"Jumlah file .txt yang diproses: {processed_files}")
    if skipped_files > 0:
        print(f"Jumlah item non-txt/subfolder yang dilewati: {skipped_files}")
    print(f"Hasil perbandingan disimpan di: {output_folder_comparison_path}") # Path output baru
    # Pesan untuk folder cleaned-only (opsional)
    if os.path.exists(output_folder_cleaned_path):
         print(f"Hasil cleaning saja disimpan di: {output_folder_cleaned_path}")
    print(f"Total waktu eksekusi: {total_time:.2f} detik")

    # Cetak Hasil Evaluasi Reduksi Kosakata TOTAL (dari semua file)
    print("\n--- Evaluasi Reduksi Kosakata TOTAL (Semua File) ---")
    count_unique_cleaned_total = len(unique_words_cleaned_total)
    count_unique_stemmed_total = len(unique_words_stemmed_total)
    print(f"Total kata (tokens) setelah cleaning (semua file): {total_words_cleaned_all_files}")
    print(f"Total kata (tokens) setelah stemming (semua file): {total_words_stemmed_all_files}")
    print(f"Jumlah kata unik (types) setelah cleaning (semua file): {count_unique_cleaned_total}")
    print(f"Jumlah kata unik (types) setelah stemming (semua file): {count_unique_stemmed_total}")

    if count_unique_cleaned_total > 0:
        reduction_percentage_total = ((count_unique_cleaned_total - count_unique_stemmed_total) / count_unique_cleaned_total) * 100
        print(f"Persentase reduksi kosakata unik TOTAL: {reduction_percentage_total:.2f}%")
    else:
        print("Tidak ada kata unik untuk menghitung reduksi total.")

except FileNotFoundError as fnf_error:
    print(f"ERROR: {fnf_error}")
    print("Pastikan folder input ada.")
except Exception as e:
    print(f"Terjadi kesalahan umum: {e}")

Menggunakan direktori kerja saat ini sebagai basis: /home/xerces/project/stemming-project
Menginisialisasi stemmer Sastrawi...
Stemmer siap.
Folder output sudah ada: /home/xerces/project/stemming-project/output/sastrawi
Folder output sudah ada: /home/xerces/project/stemming-project/output/cleaned

Memulai proses cleaning dan stemming dari folder: /home/xerces/project/stemming-project/input/indo
  -> Memproses file: Kel3_Peran Bimbingan dan Konseling Dalam Pendidikan Karakter    .txt...
  -> Memproses file: Kel6_Benarkah anak-anak butuh mata pelajaran koding dan AI di sekolah.txt...
  -> Memproses file: Dampak Tarif Resiprokal Trump terhadap Industri di Indonesia_1.txt...
  -> Memproses file: Global South dan Ilusi Netralitas_10.txt...
  -> Memproses file: Eksistensi Media Massa Nasional_5.txt...
  -> Memproses file: Peran Media Massa dalam Membentuk Opini Publik_5.txt...
  -> Memproses file: Kelompok 8_Ini 5 Bahaya Makanan Junk Food yang Perlu Diwaspadai.txt...
  -> Memproses file: Kel